In [ ]:
# !pip install tensorflow
# !pip install pandas
# !pip install sweetviz
# !pip install ydata-profiling
# !pip install ipywidgets

In [ ]:
# import sweetviz
# sweetviz.__version__

In [ ]:
import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import tensorflow as tf
import json
import torch
import segmentation_models_pytorch as smp
import os
import numpy as np
import json
# from keras.preprocessing import image
import matplotlib.pyplot as plt
from tqdm import tqdm
# import seaborn as sns
import sweetviz as sv
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
pd.__version__

In [ ]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

In [ ]:
def read_json(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    f.close()
    # Create different lists from the data dictionary

    annotations = pd.DataFrame(data["annotations"])
    category = pd.DataFrame(data["categories"])
    info = pd.DataFrame.from_dict(data["info"], orient='index')
    images = pd.DataFrame(data['images'])
    licenses = pd.DataFrame(data['licenses'])
    return annotations, category, info, images, licenses

In [ ]:
json_train_path = './dataset/fungi/train_val_annotations/train.json'
json_val_path = './dataset/fungi/train_val_annotations/val.json'
tr_annotation, tr_category, tr_info, tr_images, tr_licenses = read_json(json_train_path)
val_annotation, val_category, val_info, val_images, val_licenses = read_json(json_val_path)

In [ ]:
tr_annotation.head()

In [ ]:
tr_annotation['category_id'].nunique()

In [ ]:
categories = sorted([int(i) for i in tr_annotation['category_id'].unique().tolist()])
categories[:3]

In [ ]:
def create_merged_json_df(images, annotations, category):
    """ args = takes the three important dataframes
    returns = create a merged dataframe with all the important data
    """
    data_df = images.copy()

    ## to the images dataframe, we add the category_id column - our target
    data_df['category_id'] = annotations[annotations['image_id'] == data_df['id']]['category_id']
    category.set_index('id')
    data_df = pd.merge(left=data_df, right=category, how='left', left_on='category_id', right_on='id')
    data_df.drop(columns = ['id_y'], inplace=True)
    #change the column to a string?
    data_df['category_id'] = data_df['category_id'].astype(str) 
    # Add the columns of image name and its subdirectory/catefory
    data_df['image_name'] = data_df['file_name'].str.split('/').str[2]
    data_df['subdir'] = data_df['file_name'].str.split(pat='/', n=1).str[1]
    return data_df

In [ ]:
tr_data = create_merged_json_df(tr_images, tr_annotation, tr_category)
tr_data['freq'] = tr_data.groupby('category_id')['category_id'].transform('count')
# tr_data['category_id'] = tr_annotation[tr_annotation['image_id'] == tr_data['id']]['category_id']
# tr_data.sort_values(by=['file_name'], inplace=True)
tr_data.head(3)

In [ ]:
tr_data[tr_data['category_id']=='650'].shape

In [ ]:
category_id_train = sorted([int(i) for i in tr_data['category_id'].unique().tolist()])
category_id_train[:3]

In [ ]:
tr_data.iloc[0,1]

In [ ]:
val_data = create_merged_json_df(val_images, val_annotation, val_category)
val_data.head(3)

In [ ]:
tr_category.set_index('id').head()

In [ ]:
# images_report = sv.analyze(tr_images)
# #display the report
# images_report.show_html('sweetviz_train_images.html')

In [ ]:
# train_data_report = sv.analyze(tr_data)
# #display the report
# train_data_report.show_html('sweetviz_train_data.html')

In [ ]:
# from ydata_profiling import ProfileReport
# profile = ProfileReport(tr_images, title='images Pandas Profiling Report')
# # profile

In [ ]:
# profile.to_file('Report_train_json.html')

In [ ]:
train_images_names = tr_data['file_name'].to_list()
val_images_names = val_data['file_name'].to_list()
len(train_images_names), len(val_images_names)

In [ ]:
train_images_cat = tr_data['category_id'].to_list()
val_images_cat = val_data['category_id'].to_list()
len(train_images_cat), len(val_images_cat)

In [ ]:
train_key_value_lst = [[key, value] for key, value in zip(train_images_cat, train_images_names)]
val_key_value_lst = [[key, value] for key, value in zip(val_images_cat, val_images_names)]
train_key_value_lst[:3]

In [ ]:
train_key_value_lst[-3:]

In [ ]:
val_key_value_lst[:3]

In [ ]:
val_key_value_lst[-3:]

In [ ]:
with open('./dataset/fungi/train_images_cat.txt', 'w') as f:
    for line in train_key_value_lst:
        f.write(f"{line[0]}@:@" + "/".join(line[1].split('/')[1:]) + '\n')
f.close()

In [ ]:
with open('./dataset/fungi/val_images_cat.txt', 'w') as f:
    for line in val_key_value_lst:
        f.write(f"{line[0]}@:@" + "/".join(line[1].split('/')[1:]) + '\n')
f.close()

In [ ]:
with open('./dataset/fungi/val_images.txt', 'w') as f:
    for line in val_images_names:
        f.write("/".join(line.split('/')[1:]) + '\n')
f.close()